# Solving a mincost k-SAT problem using QAOA

Before looking at this notebook, you should understand the methods described in the qaoa_mincost_sat notebook, as we will consider those as given here.
With the methods described in the qaoa_mincost_sat notebook, we are only able to solve 2-SAT problems, because we transform our SAT problem to a QUBO problem, which is limited to quadratic terms, and then implement the corresponding Ising model in a quantum circuit.
While there are methods for quadratization of Higher Order Binary Optimization (HOBO) problems, meaning a transformation to a QUBO problem, they introduce auxiliary variables and thus require additional qubits.

The following describes, how to convert an arbitrary SAT problem to a HOBO problem and how to implement the corresponding problem circuit for applying QAOA, while using the same number of qubits as there are boolean variables in the SAT term and using a circuit depth that grows linear in the number of variables and degree of the SAT problem.

The system, we will use as an example in this notebook, is described by the following boolean formular:
$
(x_1 \vee x_2) \wedge (x_2 \vee \neg x_3 \vee x_4) \wedge (x_3 \vee \neg x_5 \vee \neg x_6)
$

We will use the same implementation cost for each feature as in the other notebook:

| Feature | Cost $c_i$ |
|---------|------------|
| $x_1$   | 30         |
| $x_2$   | 20         |
| $x_3$   | 25         |
| $x_4$   | 50         |
| $x_5$   | 10         |
| $x_6$   | 10         |

## Converting k-SAT to HOBO

For clauses containing two variables, we can just use the quadratic penalties described by [Glover](https://arxiv.org/abs/1811.11538) we used in the other notebook:

| Clause Type   | Example                        | Constraint                 | Penalty                     |
|---------------|--------------------------------|----------------------------|-----------------------------|
| No Negations  | $(x_i \vee x_j)$               | $x_i + x_j \geq 1$         | $(1 - x_i - x_j + x_i x_j)$ |
| One Negation  | $(x_i \vee \lnot{}x_j)$        | $x_i + (1-x_j) \geq 1$     | $(x_j - x_i x_j)$           |
| Two Negations | $(\lnot{}x_i \vee \lnot{}x_j)$ | $(1-x_i) + (1-x_j) \geq 1$ | $(x_i x_j)$                 |

For clauses containing $k \ge 3$ variables, we can apply the penalties recursively.

$$(x_i \vee x_j \vee x_k) = ((x_i \vee x_j) \vee x_k)$$

We first apply the quadratic penalty to $(x_i \vee x_j)$.
The resulting penalty can be seen as a new variable $x_{ij}$ that is 0 exactly when $(x_i \vee x_j)$ is fulfilled and 1 otherwise, meaning $x_{ij} = \neg(x_i \vee x_j)$
Using that, we can define the boolean funktion as

$$(x_i \vee x_j \vee x_k) = (x_k \vee \neg x_{ij})$$

which we can again transform using the given quadratic penalties as follows:

$$
(x_{ij} - x_k x_{ij})\\
= ((1 - x_i - x_j + x_i x_j) - x_k (1 - x_i - x_j + x_i x_j))\\
= (1 - x_i - x_j + x_i x_j - x_k + x_i x_k + x_j x_k - x_i x_j x_k)\\
= (1 - x_i - x_j - x_k + x_i x_j + x_i x_k + x_j x_k - x_i x_j x_k)
$$

The same principle can be applied for clauses with negations or more variables.

Using this method we can transform our example into the corresponding HOBO model.

$$
min~y(x) = (1- x_1 - x_2 + x_1 x_2) + (x_3 - x_2 x_3 - x_3 x_4 + x_2 x_3 x_4) + (x_5 x_6 - x_3 x_5 x_6)\\
~~~~~~~~~~~~~= 1 - x_1 - x_2 + x_3 + x_1 x_2 - x_2 x_3 - x_3 x_4 + x_5 x_6 + x_2 x_3 x_4 - x_3 x_5 x_6
$$
